In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [2]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [6]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [9]:
# This stops 'Run all' at this cell by causing an error
assert False

AssertionError: ignored

In [10]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Mounted at /content/gdrive


In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [12]:
%cd /content/gdrive/MyDrive/yolov4

/content/gdrive/MyDrive/yolov4


In [13]:
# this is where my datasets are stored within my Google Drive (I created a yolov4 folder to store all important files for custom training) 
!ls /mydrive/yolov4

backup	     generate_test.py	obj.names	      yolov4.conv.137
cfg	     generate_train.py	obj_plate.zip	      yolov4-obj.cfg
classes.txt  obj.data		valid_yolo_plate.zip  yolov4.weights


In [14]:
# copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
!cp /mydrive/yolov4/obj_plate.zip ../
!cp /mydrive/yolov4/valid_yolo_plate.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj_plate.zip -d data/
!unzip ../valid_yolo_plate.zip -d data/

In [ ]:
import os
import shutil

shutil.copytree('/content/gdrive/MyDrive/yolov4/data/obj_plate','/content/darknet/data/obj_plate')
shutil.copytree('/content/gdrive/MyDrive/yolov4/data/valid_yolo_plate','/content/darknet/data/valid_yolo_plate')

In [20]:
!cp /mydrive/yolov4/obj.names /content/darknet/data
!cp /mydrive/yolov4/obj.data  /content/darknet/data

In [31]:
!pwd

/content/darknet


In [32]:
%cd /content/gdrive/MyDrive/yolov4

/content/gdrive/MyDrive/yolov4


In [33]:
!python generate_train.py
!python generate_test.py

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [26]:
shutil.copyfile('/content/gdrive/MyDrive/yolov4/obj.data','/content/darknet/data/obj.data')
shutil.copyfile('/content/gdrive/MyDrive/yolov4/yolov4-obj.cfg' ,'/content/darknet/cfg/yolov4-obj.cfg')
shutil.copyfile('/content/gdrive/MyDrive/yolov4/yolov4.conv.137','/content/darknet/yolov4.conv.137')

'/content/darknet/yolov4.conv.137'

In [34]:
!pwd

/content/gdrive/MyDrive/yolov4


In [35]:
%cd /content/darknet

/content/darknet


In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
#%%capture

!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov4-obj.cfg /content/darknet/yolov4.conv.137 -dont_show -map

In [ ]:
imShow('chart.png')

In [ ]:
# kick off training from where it last saved
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov4-obj.cfg /content/gdrive/MyDrive/yolov4/backup/yolov4-obj_last.weights -dont_show

In [ ]:
!./darknet detector map /content/darknet/data/obj.data /content/darknet/cfg/yolov4-obj.cfg /content/gdrive/MyDrive/yolov4/backup/yolov4-obj_1000.weights

In [38]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' /content/darknet/cfg/yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' /content/darknet/cfg/yolov4-obj.cfg
%cd ..

/content/darknet/cfg
/content/darknet


In [39]:
from google.colab import files
files.upload()

{}

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test /content/darknet/data/obj.data /content/darknet/cfg/yolov4-obj.cfg /content/gdrive/MyDrive/yolov4/backup/yolov4-obj_1000.weights /content/gdrive/MyDrive/100832.jpg -thresh 0.3

imShow('predictions.jpg')